In [14]:
// hacky way to set java path since macos breaks this
val userhome = System.getProperty("user.home")
val jniPath = s"${userhome}/software/z3/build"
val newPath = Array(jniPath) ++  System.getProperty("java.library.path").split(":")
System.setProperty("java.library.path",newPath.distinct.mkString(":"))
//set sys_paths to null so that java.library.path will be reevaluated next time it is needed
val sysPathsField = classOf[ClassLoader].getDeclaredField("sys_paths");
sysPathsField.setAccessible(true);
sysPathsField.set(null, null);
// note: make sim link between bounder and Bounder due to macos case sensitivity before hand
val path = s"${userhome}/Documents/source/bounder/soot_hopper/target/scala-2.13/soot_hopper-assembly-0.1.jar"
//unzip -p jdiff.jar META-INF/MANIFEST.MF
interp.load.cp(os.Path(path))

import $ivy.`org.plotly-scala:plotly-almond_2.13:0.8.2`
import $ivy.`com.github.pathikrit::better-files:3.9.1`

import $ivy.`com.lihaoyi:ujson_2.13:1.3.8`
// import $ivy.`edu.colorado.plv.bounder:soot_hopper_2.13:0.1`

import plotly._, element._, layout._, Plotly._
import ujson.Value
import sys.process._

import $ivy.`com.lihaoyi::scalatags:0.12.0`

import jupyter.Displayer, jupyter.Displayers
import scala.collection.JavaConverters._
import scala.collection.mutable

userhome: String = "/home/s"
jniPath: String = "/home/s/software/z3/build"
newPath: Array[String] = Array(
  "/home/s/software/z3/build",
  "/home/s/software/z3/build",
  "",
  "/usr/java/packages/lib/amd64",
  "/usr/lib/x86_64-linux-gnu/jni",
  "/lib/x86_64-linux-gnu",
  "/usr/lib/x86_64-linux-gnu",
  "/usr/lib/jni",
  "/lib",
  "/usr/lib"
)
res13_3: String = "/home/s/software/z3/build::/usr/java/packages/lib/amd64:/usr/lib/x86_64-linux-gnu/jni:/lib/x86_64-linux-gnu:/usr/lib/x86_64-linux-gnu:/usr/lib/jni:/lib:/usr/lib"
sysPathsField: java.lang.reflect.Field = private static java.lang.String[] java.lang.ClassLoader.sys_paths
path: String = "/home/s/Documents/source/bounder/soot_hopper/target/scala-2.13/soot_hopper-assembly-0.1.jar"
import $ivy.$                                          

import $ivy.$                                         


import $ivy.$                             
// import $ivy.`edu.colorado.plv.bounder:soot_hopper_2.13:0.1`


import plotly._, element._, layout._

In [23]:
import better.files._

import scala.util.Random
import edu.colorado.plv.bounder.Driver
import edu.colorado.plv.bounder.ExpTag
import edu.colorado.plv.bounder.Action
import edu.colorado.plv.bounder.RunConfig
import edu.colorado.plv.bounder.PickleSpec
import edu.colorado.plv.bounder.lifestate.SpecSpace
import edu.colorado.plv.bounder.lifestate.SpecSignatures
import edu.colorado.plv.bounder.symbolicexecutor.state.InitialQuery
import upickle.default.read
import upickle.default.write
import scala.collection.parallel.CollectionConverters.{ImmutableSetIsParallelizable, IterableIsParallelizable}
import edu.colorado.plv.bounder.ExperimentsDb
import edu.colorado.plv.bounder.BounderUtil
import scala.concurrent.duration._
import scala.language.postfixOps
import slick.driver.H2Driver.api._
import slick.jdbc.GetResult
import slick.jdbc.SQLActionBuilder
import scala.concurrent.Await
import almond.interpreter.api.DisplayData

Driver.setZ3Path(s"${userhome}/software/z3/build")

var android_home_possible = List(s"${userhome}/Library/Android/sdk", s"${userhome}/Android/Sdk")
var android_home = android_home_possible.find(p => File(p).exists()).get             
BounderUtil.setEnv(Map("DYLD_LIBRARY_PATH" -> s"${userhome}/software/z3/build","ANDROID_HOME" -> android_home,"HOME" -> userhome))

System.setProperty("user.dir", s"${System.getProperty("user.home")}/Documents/source/bounder/notebooks/ossExp/SpecGen");
val expDir = File(System.getProperty("user.dir"))

java.library.path set to: /home/s/software/z3/build::/usr/java/packages/lib/amd64:/usr/lib/x86_64-linux-gnu/jni:/lib/x86_64-linux-gnu:/usr/lib/x86_64-linux-gnu:/usr/lib/jni:/lib:/usr/lib


: 

In [10]:
val db = new ExperimentsDb(Some(s"${userhome}/Documents/source/bounder/soot_hopper/target/scala-2.13/soot_hopper-assembly-0.1.jar"))
case class Count(n:Int)
case class StrRes(s:String)
implicit val getCountResult = GetResult(r => Count(r.<<))
implicit val getStrResult = GetResult(r => StrRes(r.<<))

//implement in a class so any sequence gets converted to a table that can be displayed in jupyter
trait TableAble {
  def headers:List[String]   
  def values:List[Any]
}

Displayers.register(classOf[Seq[TableAble]], (people: Seq[TableAble]) => {
  import scalatags.Text.all._
  Map(
    "text/html" -> {
      table(cls:="table")(
        tr(people.head.headers.map(v => th(v))),
        for (row <- people) yield tr(
          row.values.map{
              case v:String => td(v)
              case v:Int => td(v)
          }
        )
      ).render
    }
  ).asJava
})

Initializing database


db: ExperimentsDb = edu.colorado.plv.bounder.ExperimentsDb@95633d5
defined class Count
defined class StrRes
getCountResult: AnyRef with GetResult[Count] = <function1>
getStrResult: AnyRef with GetResult[StrRes] = <function1>
defined trait TableAble

In [16]:
case class JoinedResultRow(tag:String, summary:String, loc:List[String], qry:String, config:String, apk:String) extends TableAble{
    override def toString = s"JoinedResultRow(summary = ${summary}, apk = ${apk}"
    def getSummary:BounderUtil.ResultSummary = read[BounderUtil.ResultSummary](summary)
    val getConfig:RunConfig = read[RunConfig](config)
    val getInitQuery = read[InitialQuery](qry)
    assert(getConfig.initialQuery.exists{q => q == getInitQuery})
    def headers = List("tag", "summary", "qry", "apk")
    def values = List (tag  ,  summary , qry  , apk)
    def merge(other:JoinedResultRow):JoinedResultRow = {
        assert(tag == other.tag)
        assert(qry == other.qry)
        assert(config == other.config)
        assert(apk == other.apk)
        
        JoinedResultRow(tag, write[BounderUtil.ResultSummary](Driver.reduceResults(getSummary, other.getSummary)), loc ++ other.loc,qry, config, apk)
    }
}

case class SummaryRow(witnessed:Set[JoinedResultRow] = Set.empty, unreachable:Set[JoinedResultRow] = Set.empty, 
                      timeout:Set[JoinedResultRow] = Set.empty, err:Set[JoinedResultRow] = Set.empty) extends TableAble{
    def headers =  List("witnessed", "unreachable", "timeout", "err")
    def values = List(witnessed.size, unreachable.size, timeout.size, err.size)
    def add(row:JoinedResultRow):SummaryRow = {
        row.getSummary match{
            case BounderUtil.Timeout => this.copy(timeout = timeout + row)
            case BounderUtil.Witnessed => this.copy(witnessed = witnessed + row)
            case BounderUtil.Unreachable => this.copy(unreachable = unreachable + row)
            case BounderUtil.Interrupted(_) => this.copy(err = err + row)
        }
    }
}
case class ApkSummaryRow(apk:String, row:SummaryRow = SummaryRow()) extends TableAble{
    def headers = "apk"::row.headers
    def values = apk::row.values
    def add(newRow:JoinedResultRow):ApkSummaryRow = {
        assert(newRow.apk == apk)
        this.copy(row = row.add(newRow))
    }
}



implicit val getCoffeeResult = GetResult(r => JoinedResultRow(r.<<, r.<<, List(r.<<), r.<<, r.<<, r.<<))
object ReadResults{
//where cast(results.result::json->'summary' as varchar) like '%Witnessed%' or cast(results.result::json->'summary' as varchar) like '%timeout%'
    //TODO:==== figure out if this query is counting things correctly
    def selectResults() = {
        val q = sql"""select results.jobtag, cast(results.result::json->'summary' as varchar), results.loc, results.qry, jobs.config as cfg, 
        cast(jobs.config::json->'apkPath' as varchar) as benchmark from results inner join jobs on jobs.id=results.jobid"""
        Await.result(db.db.run(q.as[JoinedResultRow]), 30 seconds)   
    }
    val allResults = selectResults()
    
    val rand = new scala.util.Random
    
    // Top model
    val allTopResults = allResults.filter(res => read[ExpTag](res.tag).specRefinement == "")
    val topResultsByQuery = allTopResults.groupBy(res => (res.apk,res.qry)).map{ // each query can have multiple locations, we just merge them
        case (_, rows) => rows.reduce((a:JoinedResultRow,b:JoinedResultRow) => a.merge(b))
    }.toSeq
    val topByApp:Seq[ApkSummaryRow] = topResultsByQuery.foldLeft(Map[String,ApkSummaryRow]()){
        case (acc,joinedRow) => 
            acc + (joinedRow.apk -> acc.getOrElse(joinedRow.apk, ApkSummaryRow(joinedRow.apk)).add(joinedRow))    
    }.values.toSeq.sortBy(_.apk)
    val appList = topByApp.map{v => v.apk}
    val topByResult = Seq(topResultsByQuery.foldLeft(SummaryRow()){
        case (acc, joinedRow) => acc.add(joinedRow)
    })
    def summaryForApkTop(app:String) = {
        val res = topByApp.filter(row => row.apk.contains(app))
        assert(res.size == 1, s"Failure, wrong number of results ${res.size}")
        res.head
    }
    def sampleWitForTop(app:String) = {
        val summary = summaryForApkTop(app)
        val witnesses = summary.row.witnessed.toList
        if(witnesses.isEmpty)
            None
        else{
            val max = witnesses.size
            Some(witnesses(rand.nextInt(max)))
        }
    }
    def getTopCfgFor(app:String,clazz:String,method:String,line:Option[Int]) = {
        ???
    }
    
}
println(s"all results: ${ReadResults.allResults.size}")
println(s"all top results: ${ReadResults.allTopResults.size}")
println(s"all top results by qry: ${ReadResults.topResultsByQuery.size}")

all results: 771
all top results: 771
all top results by qry: 747


defined class JoinedResultRow
defined class SummaryRow
defined class ApkSummaryRow
getCoffeeResult: AnyRef with GetResult[JoinedResultRow] = <function1>
defined object ReadResults

In [17]:
ReadResults.topByResult

witnessed,unreachable,timeout,err
304,193,200,50


In [18]:
ReadResults.topByApp

apk,witnessed,unreachable,timeout,err
"""${baseDir}/fdroid/ch.blinkenlights.android.vanilla/1.2.2/apk/ch.blinkenlights.android.vanilla_12200.apk""",5,4,2,1
"""${baseDir}/fdroid/com.MarcosDiez.shareviahttp/2.0.14/apk/com.MarcosDiez.shareviahttp_34.apk""",0,1,0,0
"""${baseDir}/fdroid/com.android.keepass/2.6.8/apk/com.android.keepass_215.apk""",3,2,2,0
"""${baseDir}/fdroid/com.asksven.betterbatterystats/3.3.1/apk/com.asksven.betterbatterystats_107.apk""",8,1,3,0
"""${baseDir}/fdroid/com.byagowi.persiancalendar/7.8.0/apk/com.byagowi.persiancalendar_780.apk""",2,0,6,0
"""${baseDir}/fdroid/com.darshancomputing.BatteryIndicatorPro/12.0.0/apk/com.darshancomputing.BatteryIndicatorPro_26016.apk""",20,5,5,0
"""${baseDir}/fdroid/com.gpl.rpg.AndorsTrail/0.8.3/apk/com.gpl.rpg.AndorsTrail_67.apk""",6,6,11,0
"""${baseDir}/fdroid/com.nutomic.syncthingandroid/1.23.0/apk/com.nutomic.syncthingandroid_4334.apk""",3,4,2,0
"""${baseDir}/fdroid/com.owncloud.android/3.0.1/apk/com.owncloud.android_30000001.apk""",9,9,17,1
"""${baseDir}/fdroid/com.seafile.seadroid2/2.2.48/apk/com.seafile.seadroid2_123.apk""",11,22,18,0


In [27]:
ReadResults.topByApp.filter(v => v.apk.contains("org.liberty.android")).flatMap{_.row.err.toList.map{_.summary}}.foreach{println}

In [28]:
ReadResults.topByApp.filter(v => v.apk.contains("org.transdroid.lite")).flatMap{_.row.witnessed.flatMap{r => r.getConfig.initialQuery.map(v => (r.summary,v) )}}.foreach{v => println(v._1);println(write[InitialQuery](v._2))}

"\"Witnessed\""
{"t":"DisallowedCallin","className":"org.transdroid.core.seedbox.XirvikSharedSettingsActivity","methodName":"void onPreferencesChanged()","s":"{\"univQuant\":[{\"$type\":\"edu.colorado.plv.bounder.symbolicexecutor.state.NamedPureVar\",\"n\":\"t\"}],\"existQuant\":[],\"pred\":{\"$type\":\"edu.colorado.plv.bounder.lifestate.LifeState.OAbsMsg\",\"mt\":{\"$type\":\"edu.colorado.plv.bounder.ir.CIExit\"},\"signatures\":{\"$type\":\"edu.colorado.plv.bounder.lifestate.LifeState.SubClassMatcher\",\"baseSubtypeOf\":[\"android.os.AsyncTask\"],\"signatureMatcher\":\".*AsyncTask execute\\\\(.*\\\\)\",\"ident\":\"AsyncTask_execute\"},\"lsVars\":[{\"$type\":\"edu.colorado.plv.bounder.symbolicexecutor.state.TopVal\"},{\"$type\":\"edu.colorado.plv.bounder.symbolicexecutor.state.NamedPureVar\",\"n\":\"t\"}]},\"target\":{\"$type\":\"edu.colorado.plv.bounder.lifestate.LifeState.OAbsMsg\",\"mt\":{\"$type\":\"edu.colorado.plv.bounder.ir.CIEnter\"},\"signatures\":{\"$type\":\"edu.colorado.plv

In [21]:
// TODO: see if we can validate that the same alarms exist
object SamplesToSpecify{
    // choose 1 witness sample per app
    def toInt(s: String): Option[Int] = {
      try {
        Some(s.toInt)
      } catch {
        case e: Exception => None
      }
    }
    def createHeuristicDir(cfg:RunConfig) = {
        val appDirName:String = cfg.outFolder.get.split("/").last
        val appDir = expDir / appDirName
        appDir.createIfNotExists(asDirectory=true)
        val heuristicDir = appDir / cfg.tag.heuristicType
        heuristicDir.createIfNotExists(asDirectory=true)
        heuristicDir
    }
    def writeSamples(count:Int) = {
        // val samples = ReadResults.appList.map(app => ReadResults.sampleWitForTop(app)) //TODO: random choice needs to be moved so it gets more than one per app per run
        val samples = ReadResults.topByApp.flatMap{appRes => 
            Random.shuffle(appRes.row.witnessed).take(count)
        }
        samples.zipWithIndex.foreach{
            case (sample,ind) => 
                val cfg = sample.getConfig
                val heuristicDir = createHeuristicDir(cfg)
                val existingDir = heuristicDir.glob("*").toList.filter(a => !a.name.startsWith("."))
                val existingConfigs = heuristicDir.glob("**/config.json").toList.map(v => read[RunConfig](v.contentAsString))
                val currentCfg = read[RunConfig](sample.config).copy(initialQuery=List(read[InitialQuery](sample.qry)))
                if(existingDir.size < count){
                    if(existingConfigs.forall(_ != currentCfg)){
                        val nextNum = if(existingDir.isEmpty) 0 else{ existingDir.flatMap(v => toInt(v.name)).max + 1 }
                        val sampleDir = heuristicDir / nextNum.toString
                        sampleDir.createIfNotExists(asDirectory=true)
                        (sampleDir / "config.json").overwrite(write[RunConfig](currentCfg))
                    }
                }
                
                // val f = expDir / s"sample_${ind}"
                // assert(!f.exists())
                // f.createIfNotExists(asDirectory=true)
                // (f / "apk").overwrite(sample.apk)
                // (f / "summary").overwrite(sample.summary)
                // (f / "configTop.json").overwrite(write[RunConfig](read[RunConfig](sample.config).copy(initialQuery=List(read[InitialQuery](sample.qry)))))
        }
    }
}
SamplesToSpecify.writeSamples(1)
println(s"Samples to specify: ${expDir.glob("**/configTop.json").toList.size}")

Samples to specify: 0


defined object SamplesToSpecify

In [47]:
import edu.colorado.plv.bounder.lifestate.FragmentGetActivityNullSpec
import edu.colorado.plv.bounder.lifestate.LifeState.{AbsMsg, LSSpec, LSPred, OAbsMsg, SetSignatureMatcher, Signature, SignatureMatcher, SubClassMatcher,Or,And,NS,Not}
import edu.colorado.plv.bounder.symbolicexecutor.state.{BoolVal, Equals, NamedPureVar, NotEquals, NullVal, PureExpr, TopVal}


import edu.colorado.plv.bounder.ir.{CBEnter, CBExit}

object ToSpecify{
    val allCfg = expDir.glob("**/config.json").map{f => (f,read[RunConfig](f.contentAsString))}.toList
}
println(ToSpecify.allCfg.size)

29


import edu.colorado.plv.bounder.lifestate.FragmentGetActivityNullSpec

import edu.colorado.plv.bounder.lifestate.LifeState.{AbsMsg, LSSpec, LSPred, OAbsMsg, SetSignatureMatcher, Signature, SignatureMatcher, SubClassMatcher,Or,And,NS,Not}

import edu.colorado.plv.bounder.symbolicexecutor.state.{BoolVal, Equals, NamedPureVar, NotEquals, NullVal, PureExpr, TopVal}



import edu.colorado.plv.bounder.ir.{CBEnter, CBExit}


defined object ToSpecify

In [47]:
//ch.blinkenlights.android  (20 min 3 seconds so far) + 
// note: above pauses were checking other unrelated issues with historia implementation
object Specify_0{
    val initial = read[InitialQuery]("""
        {
          "methodName": "void onClick(android.content.DialogInterface,int)",
          "line": 297,
          "t": "ReceiverNonNull",
          "matcher": "void onBackPressed()",
          "className": "ch.blinkenlights.android.vanilla.PreferencesActivity$HeadsetLaunchFragment$2"
        }
      """)

    private val mConfig = ToSpecify.allCfg.filter{case (_,cfg) => cfg.initialQuery.exists(v => v == initial)}
    assert(mConfig.size == 1)
    val config = mConfig.head._2
    val configFile = mConfig.head._1
    
    val Fragment_onCreate: SignatureMatcher = SubClassMatcher(SpecSignatures.Fragment,
          "void onCreate\\(android.os.Bundle\\)", "Fragment_onCreate")
                                                                                                             
    val Fragment_onCreate_entry: OAbsMsg = AbsMsg(CBEnter, Fragment_onCreate, TopVal::NamedPureVar("f")::Nil)
    val fragmentActivityNotAttached_improved: LSPred =
        Or(And(
            NS(SpecSignatures.Fragment_onDestroy_exit, SpecSignatures.Fragment_onActivityCreated_entry),
            NS(SpecSignatures.Fragment_onDestroy_exit, Fragment_onCreate_entry)
        ),
          And(Not(SpecSignatures.Fragment_onActivityCreated_entry), Fragment_onCreate_entry))
    val configWithSpec = config.copy(specSet = config.specSet.asInstanceOf[PickleSpec].copy(specs = Set(FragmentGetActivityNullSpec.getActivityNonNull)))
    def writeSpec():Unit = {
        val outf = File(configFile.toString + "_spec.json")
        outf.overwrite(write(configWithSpec))
    }
}
Specify_0.writeSpec()

cmd47.sc:17: value Fragment in object SpecSignatures cannot be accessed as a member of object edu.colorado.plv.bounder.lifestate.SpecSignatures from object Specify_0 in object Helper
    val Fragment_onCreate: SignatureMatcher = SubClassMatcher(SpecSignatures.Fragment,
                                                                             ^Compilation Failed

: 